In [1]:
import pandas as pd, numpy as np

# ArcGIS in Python
*Adapted from 2022 Summer CSS*

Today we are going to try using ArcGIS in Python!

## Setting Up
To begin we will need to install arcgis

In [2]:
# installing arcgis library
# this may take some time depending on how many dependencies are needed
!pip install arcgis


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# we will also need to ensure we are using a compatible ipython widget
!pip install ipywidgets


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


When you are using a Python script to connect to your UCSD linked ArcGIS Online account, the authentication is slightly different though as you do not directly pass in your SSO info.

![SSO Login](sso.png)

Instead you will need to follow the steps listed below in your ArcGIS Online account to generate a unique application ID that can be used to prove your identify. The steps below show how a client id can be obtained by registering a new application with your GIS. Only one client id is required, so if your GIS has one already, you may use it instead of creating a new application.

1. Log into your UCSD ArcGIS Online account
2. Click on the Content tab
3. Click + New Item near the top right of the page. In the popup window that appears click Application to specify the type of item you want to create. On the next page in the popup window, select the button for Other Application then click next. On the final page in the popup window fill in the information below.
    - Title: Python
    - Tags: python
    - Summary: python
4. On the Item details page of this newly created application, navigate to the Settings tab
5. Click the Registered Info (or register application) button located near the bottom of the page. This will dispaly an App ID. This is your client id and needs to be passed in as the client_id parameter when construcing a GIS object. You need this in your Python code to log in.
    - Save this ID elsewhere to make things easier!
    - Stay signed in to ArcGIS! We will need it open for this last part.
Once you have followed the process and generated a Client ID in ArcGIS Online, use the following Python code in your script but update the test_client_id string to reflect the ID you obtain by following the directions in the API documentation:
    - You will be prompted to insert a code along with a webpage opening, simply click allow and copy the code into the prompt.
Keep in mind that if you are working on a PC in the standard Python prompt with the Command Prompt application, CTRL-V may not work for pasting in the copied code and you may receive an error about the code being incorrect if you try this approach. Instead just use your mouse to right click in the prompt after you’ve made sure that you’ve copied the authentication code, and then hit Enter on your keyboard – the right click operation should successfully paste in the code and allow the rest of the script to run successfully. If the authentication process is successful the final line of the code provided above should print your user information confirming that the script was able to access your account information without any problems.

In [7]:
import arcgis
# test_client_id = 'PASTE_CLIENT_ID_HERE'

# Note, you can make things simpler for GIS objects by using
from arcgis.gis import GIS
gis = GIS('https://ucsdonline.maps.arcgis.com', client_id=test_client_id)
print(gis.users.me)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=sBqLj0OKfuK5ef1O&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=j4MNCWyqb7SZgySYEebhRRSb0jQeg3&allow_verification=false


/Users/Andrew/Documents/Projects/Code/Python/Python/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<User username:aalona_UCSDOnline>


If you see something similar to ```<User username:sstudent_UCSDOnline>``` with your own username, congrats! You have connected ArcGIS to Python and are ready for mapping!

**From now on you'll only need the code block above to use GIS in Python**

## Lab 2  – GIS and Public health (COVID dashboards and other health trend maps and analysis)

This lab aims to introduce you to how to download COVID data, insert spatial data into the program, and utilize attribute queries and joins in a map. This will allow us to look at COVID data in comparison to vulnerable elderly populations. We will cover this lab together step-by-step and this outline below covers what we learned today! Each lab tutorial will end with a final map that will have required you to use several skills within GIS to create.

Task: *You are interested in identifying the most susceptible counties in America to COVID-19. Using two layers, we will make a basic map to start this task.*

### Initialize your map

This is to create your new map, think of this as an empty canvas. It will look

In [8]:
# Create a GIS map
map1 = gis.map()


### Download and load data:

We'll be using the [COVID-19 weekly deaths data](https://data.cdc.gov/Case-Surveillance/Weekly-United-States-COVID-19-Cases-and-Deaths-by-/pwn4-m3yp) in tabular format.

We are going to filter the data so it only is for the most recent day within the date column. After downloading the table in CSV format, check the data and make sure that the table looks good for joining (proper data that we need, all columns have zero spaces).

In [26]:
COVID_deaths_df = pd.read_csv("Weekly_United_States_COVID-19_Cases_and_Deaths_by_State_-_ARCHIVED.csv")
COVID_deaths_df.head()
# always make sure your data is ready before working on analyses!

,date_updated,state,start_date,end_date,tot_cases,new_cases,tot_deaths,new_deaths,new_historic_cases,new_historic_deaths
0,01/23/2020,AK,01/16/2020,01/22/2020,0,0,0,0,0,0
1,01/30/2020,AK,01/23/2020,01/29/2020,0,0,0,0,0,0
2,02/06/2020,AK,01/30/2020,02/05/2020,0,0,0,0,0,0
3,02/13/2020,AK,02/06/2020,02/12/2020,0,0,0,0,0,0
4,02/20/2020,AK,02/13/2020,02/19/2020,0,0,0,0,0,0


Next, we need to use a shapefile to join our COVID table to. In this case, we are looking at the State level, so we will utilize a dataset for state boundaries in the USA taken from [Here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html).
   - Because we are using arcgis online, we will need to upload or **publish** that local file.

In [48]:
# add in the name of your shapefile here
#shapefile_path = r'path\to\your\shapefile\shapefile_name.zip'
shapefile_location = 'cb_2018_us_state_20m.zip'

# these properties are the exact same as you see in ArcGIS Online, just as an object
shapefile_properties = {
    'title': 'United States States Boundary',
    'tags': 'shapefile, United States',
    'type': 'Shapefile',
    'access': 'private'
}

# Upload shapefile
shapefile_states_item = gis.content.add(shapefile_properties, data=shapefile_location)
# Access uploaded file
shapefile_states_item

# Publishing
published_states_feature_layer_item = shapefile_states_item.publish()
published_states_feature_layer_item

<Item title:"United States States Boundary" type:Feature Layer Collection owner:aalona_UCSDOnline>

Having an instant publish setup is very powerful as you could upload a list of multiple shapefiles and more to ArcGIS Online!

Here's all possible parameters for item_properties:

    'title': The title of the item.
    'type': The type of the item (e.g., 'Shapefile', 'CSV', 'Feature Service', etc.).
    'tags': Comma-separated list of tags associated with the item.
    'snippet': A short description or summary of the item.
    'description': A detailed description of the item.
    'access': The access level of the item ('public', 'private', 'shared').
    'licenseInfo': Information about the item's licensing.
    'culture': The culture/locale of the item's content (e.g., 'en-us').
    'thumbnail': The URL or file path to a thumbnail image for the item.


Now let's do the same for our csv!

In [49]:
# csv_path = r'path\to\your\csv\csv_name.csv'
csv_file = 'Weekly_United_States_COVID-19_Cases_and_Deaths_by_State_-_ARCHIVED.csv'

# Define item properties for CSV
States_properties = {
    'title': 'COVID-19 Deaths by State',
    'tags': 'COVID-19, deaths, CSV',
    'type': 'CSV',
    'access': 'private'
}

# Upload CSV file
csv_COVID_deaths_item = gis.content.add(States_properties, csv_file)
# Then Publish said file
published_csv_COVID_death = csv_COVID_deaths_item.publish()

# Access uploaded file
published_csv_COVID_death

<Item title:"COVID-19 Deaths by State" type:Feature Layer Collection owner:aalona_UCSDOnline>

What you've just uploaded is fully accessibly from ArcGIS Online. If you ever need to access the files immediately using your browser, you can simple use ```.url``` to find the link.

If you no longer have the object loaded, why not try searching for it like the following?

In [55]:
query = 'title: "YOUR PUBLISHED TITLE HERE"'
search_results = gis.content.search(query=query, max_items=10)
search_results

[<Item title:"United States States Boundary" type:Feature Layer Collection owner:aalona_UCSDOnline>,
 <Item title:"United States States Boundary" type:Shapefile owner:aalona_UCSDOnline>,
 <Item title:"Aggregation_of_mclocations_to_United_States_State_Boundaries_2018" type:Feature Layer Collection owner:abchin_dsc170fa20>]

Try to find the correct item of your States Boundaries using slicing and load it in.
Hint: We are looking for feature layers, aka the published shapefile.

In [56]:
# Try finding the correct item using slicing
published_states_feature_layer_item = search_results[99] # FIX HERE
published_states_feature_layer_item.url

'https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/Aggregation_of_mclocations_to_United_States_State_Boundaries_2018/FeatureServer'

In [57]:
# You can even extract individual layers from your layer items
feature_layers = published_states_feature_layer_item.layers
states_layer = feature_layers[0]
# in this case we only have 1
states_layer.properties.name


'Aggregation_of_mclocations_to_United_States_State_Boundaries_2018'

Let's go ahead and upload our 65 and over census data.

In [ ]:
csv_path = 'FIX HERE'

# Define item properties for CSV
csv_properties = {
'FILL IN HERE'
}

# Upload CSV file
uploaded_csv_elderly_population = # FIX HERE

# Then Publish said file
published_csv_elderly_population = # FIX HERE

# Access uploaded file
published_csv_elderly_population

And that's it for now! We can get back to mapping in online ArcGIS.

Later we will go over how to map and edit layers in Python itself!